# Simulation of Quantum Key Distribution and Eavesdropper

This note confirms previous evaluation results by simulating quantum key distribution protocol.

## Contents

1. Without eavesdropper
2. With eavesdropper using ordinary measurement method
3. With eavesdropper using another measurement method (eavesdropper with angle)

To evaluate the performance, P(Alice == Bob) and P(Bob == Eve) are calculated.

- P(Alice == Bob) is the probability that Alice's bit is equal to Bob's basis.
- P(Bob == Eve) is the probability that Alice's bit is equal to Eve's basis.

For details, see qkd.ipynb.

This note uses some codes from a qiskit tutorial, "quantum-key-distribution.ipynb".

In [1]:
# import
import numpy as np
import itertools
from math import pi
from numpy.random import randint
import matplotlib.pyplot as plt

#%matplotlib inline

from qiskit import (
    QuantumCircuit,
    ClassicalRegister,
    QuantumRegister,
    execute,
    BasicAer,
    Aer,
    transpile,
    assemble,
)
from qiskit.visualization import plot_histogram, plot_bloch_multivector

print("Imports Successful")


Imports Successful


## 1. Without eavesdropper

In [2]:
def calc_theta(p):
    return 2 * np.arccos(np.sqrt(p))


def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits


In [3]:
def encode_message_without_intercept(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1, 1)
        if bases[i] == 0:  # prepare qubit in Z-basis
            if bits[i] == 0:
                pass
            else:
                qc.x(0)
        else:  # prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message


def measure_message_without_intercept(message, bases):
    backend = Aer.get_backend("aer_simulator")
    measurements = []
    for q in range(n):
        if bases[q] == 0:  # measuring in Z-basis
            message[q].measure(0, 0)
        else:  # measuring in X-basis
            message[q].h(0)
            message[q].measure(0, 0)
        aer_sim = Aer.get_backend("aer_simulator")
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements


In [4]:
# without interception

n = 4000
## Step 1
# Alice generates bits
alice_bits = randint(2, size=n)
# print(f'alice_bits={alice_bits}')

## Step 2
# Create an array to tell us which qubits
# are encoded in which bases
alice_bases = randint(2, size=n)
# print(f'alice_bases={alice_bases}')
message = encode_message_without_intercept(alice_bits, alice_bases)

## Step 3
# Decide which basis to measure in:
bob_bases = randint(2, size=n)
# print(f'bob_bases={bob_bases}')
bob_results = measure_message_without_intercept(message, bob_bases)
# print(f'bob_results={bob_results}')

## Step 4
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
# print(f'alice_key={alice_key}')
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
# print(f'bob_key={bob_key}')


Simulation result shows that P(Alice == Bob) = 1 as expected.

In [5]:
bob_key = np.array(bob_key)
alice_key = np.array(alice_key)

print(
    f"prob(alice==bob)={np.sum(bob_key == alice_key)/len(alice_key):0.2f}, len(alice_key)={len(alice_key)}"
)


prob(alice==bob)=1.00, len(alice_key)=2002


## 2. With eavesdropper using ordinary measurement method

In [6]:
def encode_message_with_intercept(bits, alice_bases, eve_bases, bob_bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1, 2)
        if alice_bases[i] == 0:  # prepare qubit in Z-basis
            if bits[i] == 0:
                pass
            else:
                qc.x(0)
        else:  # prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)

        if eve_bases[i] == 0:  # measuring in Z-basis
            qc.measure(0, 0)
        else:  # measuring in X-basis
            qc.h(0)
            qc.measure(0, 0)
            qc.h(0)

        if bob_bases[i] == 0:  # measuring in Z-basis
            qc.measure(0, 1)
        else:  # measuring in X-basis
            qc.h(0)
            qc.measure(0, 1)

        message.append(qc)
    return message


def measure_message_with_intercept(message):
    backend = Aer.get_backend("aer_simulator")
    measurements_eve = []
    measurements_bob = []
    for q in range(n):
        aer_sim = Aer.get_backend("aer_simulator")
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = result.get_memory()[0]
        measurements_eve.append(int(measured_bit[1]))
        measurements_bob.append(int(measured_bit[0]))
    return measurements_eve, measurements_bob


In [7]:
# with interception

n = 4000
# Step 1
alice_bits = randint(2, size=n)
alice_bases = randint(2, size=n)
eve_bases = randint(2, size=n)
bob_bases = randint(2, size=n)

# Step 2
message = encode_message_with_intercept(alice_bits, alice_bases, eve_bases, bob_bases)

# Step 3
eve_results, bob_results = measure_message_with_intercept(message)

# Step 4
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
eve_key = remove_garbage(alice_bases, bob_bases, eve_results)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)


Simulation results show that P(Alice == Bob) $\sim$ 0.75 and P(Bob == Eve) $\sim$ 0.75 as expected.

In [8]:
bob_key = np.array(bob_key)
eve_key = np.array(eve_key)
alice_key = np.array(alice_key)

print(
    f"prob(alice==bob)={np.sum(bob_key == alice_key)/len(alice_key):0.2f}, len(alice_key)={len(alice_key)}"
)
print(
    f"prob(bob==eve)={np.sum(eve_key == bob_key)/len(bob_key):0.2f}, len(bob_key)={len(bob_key)}"
)


prob(alice==bob)=0.75, len(alice_key)=1974
prob(bob==eve)=0.74, len(bob_key)=1974


## 3. With eavesdropper using another measurement method (eavesdropper with angle)

In [9]:
def encode_message_with_mod_intercept(p, angle, bits, alice_bases, bob_bases):
    message = []
    theta = calc_theta(p)
    for i in range(n):
        qc = QuantumCircuit(3, 2)
        qc.u(theta, 0, 0, 0)

        # Alice
        if alice_bases[i] == 0:  # prepare qubit in Z-basis
            if bits[i] == 0:
                pass
            else:
                qc.x(1)
        else:  # prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(1)
            else:
                qc.x(1)
                qc.h(1)

        # Eve
        qc.cswap(0, 1, 2)
        qc.u(angle, 0, pi, 1)
        qc.measure(1, 0)
        qc.u(angle, 0, pi, 1)
        qc.cswap(0, 1, 2)

        # Bob
        if bob_bases[i] == 0:  # measuring in Z-basis
            qc.measure(1, 1)
        else:  # measuring in X-basis
            qc.h(1)
            qc.measure(1, 1)

        message.append(qc)
    return message


def measure_message_with_mod_intercept(message):
    backend = Aer.get_backend("aer_simulator")
    measurements_eve = []
    measurements_bob = []
    for q in range(n):
        aer_sim = Aer.get_backend("aer_simulator")
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = result.get_memory()[0]
        measurements_eve.append(int(measured_bit[1]))
        measurements_bob.append(int(measured_bit[0]))
    return measurements_eve, measurements_bob


In [10]:
# with modified interception

n = 4000
# Step 1
alice_bits = randint(2, size=n)
alice_bases = randint(2, size=n)
bob_bases = randint(2, size=n)

# Step 2: swap p = 0.2, theta = pi/4
message = encode_message_with_mod_intercept(
    1 - 0.2, pi / 4, alice_bits, alice_bases, bob_bases
)

# Step 3
eve_results, bob_results = measure_message_with_mod_intercept(message)

# Step 4
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
eve_key = remove_garbage(alice_bases, bob_bases, eve_results)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)


Simulation results agree with the evaluation results that P(Alice==Bob) $\sim$ 0.8 and P(Eve==Bob) $\sim$ 0.78 when theta is pi/4 and swap p is 0.2.

In [11]:
bob_key = np.array(bob_key)
eve_key = np.array(eve_key)
alice_key = np.array(alice_key)

print(
    f"prob(alice==bob)={np.sum(bob_key == alice_key)/len(alice_key):0.2f}, len(alice_key)={len(alice_key)}"
)
print(
    f"prob(bob==eve)={np.sum(eve_key == bob_key)/len(bob_key):0.2f}, len(bob_key)={len(bob_key)}"
)


prob(alice==bob)=0.80, len(alice_key)=2007
prob(bob==eve)=0.79, len(bob_key)=2007
